In [1]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [2]:
movies = pd.read_csv('C:\\jupyter\\recommended system\\ml-latest\\movies.csv')
ratings = pd.read_csv('C:\\jupyter\\recommended system\\ml-latest\\ratings.csv')

In [3]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi


In [4]:
ratings

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
27753439,283228,8542,4.5,1379882795
27753440,283228,8712,4.5,1379882751
27753441,283228,34405,4.5,1379882889
27753442,283228,44761,4.5,1354159524


In [37]:
#на полном датасете не получилось сделать рекомендацию, возьмем только первые 100 000
r = ratings[:100000]
r

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
99995,1041,1393,4.0,939449433
99996,1041,1411,4.0,939449551
99997,1041,1419,3.0,939447914
99998,1041,1446,4.0,939448272


In [29]:
%%time
#создаем общую таблицу из которой понятно какой пользователь какому фильму поставил какую оценку
movies_with_ratings = movies.join(r.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

Wall time: 57.9 ms


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,4.0,1.113766e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,5.0,9.488858e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,1.442169e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,4.0,1.370810e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,22.0,4.0,1.237623e+09


In [30]:
movies_with_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,4.0,1.113766e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,5.0,9.488858e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,1.442169e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,4.0,1.370810e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,22.0,4.0,1.237623e+09
...,...,...,...,...,...,...
148106,192081,Like Father (2018),Comedy,371.0,3.5,1.533570e+09
148163,192215,Forbidden Films (2014),Documentary,371.0,2.5,1.533570e+09
148165,192219,My Big Gay Italian Wedding (2018),Comedy,371.0,1.5,1.533570e+09
148168,192225,Redwall The Movie (2000),Animation|Children|Comedy|Fantasy,371.0,4.0,1.533570e+09


In [31]:
%%time
#создаем датасет для surprise
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

Wall time: 2 ms


In [32]:
dataset

,uid,iid,rating
0,4.0,Toy Story (1995),4.0
1,10.0,Toy Story (1995),5.0
2,14.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),4.0
4,22.0,Toy Story (1995),4.0
...,...,...,...
148106,371.0,Like Father (2018),3.5
148163,371.0,Forbidden Films (2014),2.5
148165,371.0,My Big Gay Italian Wedding (2018),1.5
148168,371.0,Redwall The Movie (2000),4.0


In [33]:
%%time
#создаем данные для surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

Wall time: 79.8 ms


In [34]:
%%time
trainset, testset = train_test_split(data, test_size=0.20, random_state = 40)

Wall time: 98.2 ms


In [35]:
%%time
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Wall time: 8.81 s


In [36]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred)

RMSE: 0.9684


0.9683679151911267

### Попробуем различные алгоритмы для построения рекомендательной системы

**KNNWithMeans**

In [40]:
%%time
#проверим разные similarities module при KNNWithMeans
similarities = ['msd', 'cosine', 'pearson', 'pearson_baseline']

for sim in similarities: 
    algo = KNNWithMeans(k=50, sim_options={'name': sim, 'user_based': True})
    algo.fit(trainset)
    test_pred = algo.test(testset)
    print(f'при {sim} rmse = {accuracy.rmse(test_pred)} \n' )

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9723
при msd rmse = 0.9722772122705085 

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9769
при cosine rmse = 0.9769368439751244 

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9791
при pearson rmse = 0.9791453493907026 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9778
при pearson_baseline rmse = 0.9777519646330288 

Wall time: 7.91 s


**Лучше всего резульаты получились при использовании "pearson" в качестве similarity matrix**

In [45]:
%%time
#проверим user-based параметр
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson', 'user_based': False})
algo.fit(trainset)
test_pred = algo.test(testset)
print(accuracy.rmse(test_pred))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9921
0.9921320486406628
Wall time: 22.2 s


**Вероятнее всего из-за ограниченного объема выборки при user_based = False результаты получаются выше**

In [49]:
%%time
#проверим разные К
k_list = [30, 40, 50, 75]

for k in k_list: 
    algo = KNNWithMeans(k = k, sim_options={'name': 'pearson', 'user_based': False})
    algo.fit(trainset)
    test_pred = algo.test(testset)
    print(f'при k = {k} rmse = {accuracy.rmse(test_pred)} \n' )

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9975
при k = 30 rmse = 0.9975097840702514 

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9939
при k = 40 rmse = 0.9938630067749866 

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9921
при k = 50 rmse = 0.9921320486406628 

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9890
при k = 75 rmse = 0.9889734264650542 

Wall time: 1min 28s


**Лучше всего результаты получились при k = 30**

#### Таким образом, наилучшие параметры алгоритма KNNWithMeans k = 30, similarity = pearson, user_based = False.